In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tifffile as tiff

In [2]:
import glob
import numpy as np
def get_dataset(X1_dir, X2_dir,Y_dir):
    ms_pan=[]
    gt=[]
    ms_pan_t = []
    gt_t = []
    for d in range(len(X1_dir)):
        ms1 = tiff.imread(X1_dir[d])
        pan= tiff.imread(X2_dir[d])
        gt1 = tiff.imread(Y_dir[d])
        # print(ms1.shape,pan.shape,gt1.shape)


        pan=np.expand_dims(pan,axis=-1)
        m,n,c=ms1.shape
        M,N,C=pan.shape

        pan_lr= downsample_bicubic(pan,4)
        ms_lr=downsample_bicubic(ms1,4)
        # print(ms_lr.shape)
        pan_lr=np.expand_dims(pan_lr,axis=-1)
        u_ms=upsample_bicubic(ms_lr,4)
        # print(pan_lr.shape)
        # print(u_ms.shape)
        up_ch= np.concatenate((pan_lr,u_ms),axis=-1)
        up_ch= np.expand_dims(up_ch,axis=0)


        patch_size=64; overlap=32
        x_train=tf.image.extract_patches(images=up_ch,sizes=(1,patch_size,patch_size,1),strides=(1,overlap,overlap,1),rates=(1,1,1,1),padding='VALID')
        x_train=np.reshape(x_train,(-1,patch_size,patch_size,5))
        x_train=x_train.astype('float32')
        # print(x_train.shape)

        y_train=np.expand_dims(gt1,axis=0)
        y_train=tf.image.extract_patches(images=y_train,sizes=(1,patch_size,patch_size,1),strides=(1,overlap,overlap,1),rates=(1,1,1,1),padding='VALID')
        y_train=np.reshape(y_train,(-1,patch_size,patch_size,4))
        y_train=y_train.astype('float32')
        # print(y_train.shape)
        if d==4:
          for i in range(y_train.shape[0]):
            gt_t.append(y_train[i])
            ms_pan_t.append(x_train[i])
        else:
          for i in range(y_train.shape[0]):
            gt.append(y_train[i])
            ms_pan.append(x_train[i])

    return ms_pan,gt,gt_t,ms_pan_t

X1_train = glob.glob("/content/drive/MyDrive/dataset/PAirMax/PAirMax/FR/MS_LR/*.tif")
X2_train = glob.glob("/content/drive/MyDrive/dataset/PAirMax/PAirMax/FR/PAN/*.tif")
y_train = glob.glob("/content/drive/MyDrive/dataset/PAirMax/PAirMax/GT/*.tif")

X1_train.sort()
X2_train.sort()
y_train.sort()

x_train, y_train, x_test, y_test = get_dataset(X1_train,X2_train, y_train)

x_train=np.array(x_train)
y_train=np.array(y_train)
print(x_train.shape)
print(y_train.shape)

x_test=np.array(x_test)
y_test=np.array(y_test)
print(x_test.shape)
print(y_test.shape)

(0,)
(0,)
(0,)
(0,)
